# Instant-ngp 

This notebook aims to be a step-by-step guide to train NeRF models and rendering videos from them with nvidia's [instant-ngp](https://github.com/NVlabs/instant-ngp) software using:
 * **Colab** for the heavy lifting.
 * A low-resource **local computer** for the steps that require having a graphical user interface (GUI).

It has been tested on a GTX 1050ti in the local machine and an assigned Tesla T4 in the remote one.

Based on this [notebook](https://colab.research.google.com/drive/10TgQ4gyVejlHiinrmm5XOvQQmgVziK3i?usp=sharing) by [@myagues](https://github.com/NVlabs/instant-ngp/issues/6#issuecomment-1016397579), the main differences being the addition of steps 3 and 4 to ensure compatibility between the local machine and the models trained in the remote machine, of step 10 to render a video from the scene, and a more guided approach.

## 1.Connect to a GPU runtime

Connect your colab session to a GPU runtime and check that you have been assigned a GPU. It should have a minimum of 8GB of available memory.

In [1]:
!nvidia-smi

Sun Mar  5 19:43:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.19       Driver Version: 527.19       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:06:00.0  On |                  N/A |
|  0%   56C    P0    N/A / 120W |    548MiB /  4096MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ... WDDM  | 00000000:07:00.0  On |                  N/A |
|  0%   

## 2. Install dependencies and clone the instant-ngp repo

In [2]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!pip install --upgrade cmake

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp

Submodule path 'dependencies/OpenXR-SDK': checked out 'e2da9ce83a4388c9622da328bf48548471261290'
Submodule path 'dependencies/args': checked out 'a48e1f880813b367d2354963a58dedbf2b708584'
Submodule path 'dependencies/dlss': checked out 'b3559040f968d26b2edbe37e8e3ceda4b8d1275a'
Submodule path 'dependencies/dlss/NVIDIAImageScaling': checked out '4d3f6f1b421dda208b937f2ac78657c6753efccd'
Submodule path 'dependencies/glfw': checked out '71eb7036b47e2733c2f7b4c0010e2ce62557009d'
Submodule path 'dependencies/glm': checked out 'efec5db081e3aad807d0731e172ac597f6a39447'
Submodule path 'dependencies/imgui': checked out 'fa2b318dd6190852a6fe7ebc952b6551e93899e0'
Submodule path 'dependencies/pybind11': checked out '7a5068336979377fbf4aa66bbaa483c4cb1c76a7'
Submodule path 'dependencies/tiny-cuda-nn': checked out '44f10182520fed8cd744113948045ee7ab1bacb5'
Submodule path 'dependencies/tiny-cuda-nn/dependencies/cutlass': checked out '1eb6355182a5124639ce9d3ff165732a94ed9a70'
Submodule path 'dependen

Cloning into 'instant-ngp'...
Submodule 'dependencies/OpenXR-SDK' (https://github.com/KhronosGroup/OpenXR-SDK.git) registered for path 'dependencies/OpenXR-SDK'
Submodule 'dependencies/args' (https://github.com/Taywee/args) registered for path 'dependencies/args'
Submodule 'dependencies/dlss' (https://github.com/NVIDIA/DLSS) registered for path 'dependencies/dlss'
Submodule 'dependencies/glfw' (https://github.com/Tom94/glfw) registered for path 'dependencies/glfw'
Submodule 'dependencies/glm' (https://github.com/g-truc/glm) registered for path 'dependencies/glm'
Submodule 'dependencies/imgui' (https://github.com/ocornut/imgui.git) registered for path 'dependencies/imgui'
Submodule 'dependencies/pybind11' (https://github.com/Tom94/pybind11) registered for path 'dependencies/pybind11'
Submodule 'dependencies/tiny-cuda-nn' (https://github.com/NVlabs/tiny-cuda-nn) registered for path 'dependencies/tiny-cuda-nn'
Submodule 'dependencies/tinylogger' (https://github.com/Tom94/tinylogger) regis

## 3. Set compute capability
Find the compute capability of the GPU in your **local** machine in the following link:
https://developer.nvidia.com/cuda-gpus

You need this to be able to open your trained models in `testbed` inside your local machine later on, so you can explore them or trace a camera path in order to generate a video from your scene.

In [4]:
compute_capability = "61" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability


env: TCNN_CUDA_ARCHITECTURES=61


## 4. Set the right network configuration
For compatibility between the model trained here and the local machine, a network with FP32 or FP16 is chosen.

https://docs.nvidia.com/deeplearning/tensorrt/support-matrix/index.html#hardware-precision-matrix 

In [5]:
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH

Using CutlassMLP
env: NN_CONFIG_PATH=./configs/nerf/base.json


'jq' is not recognized as an internal or external command,
operable program or batch file.


## 5. Build the project and install python requirements

In [6]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF

-- Building for: NMake Makefiles
-- Configuring incomplete, errors occurred!
See also "E:/Github/band-gap/instant-ngp/build/CMakeFiles/CMakeOutput.log".


CMake Error at CMakeLists.txt:11 (project):
  Running

   'nmake' '-?'

  failed with:

   The system cannot find the file specified


CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
CMake Error: CMAKE_CUDA_COMPILER not set, after EnableLanguage


In [7]:
!cmake --build build --config RelWithDebInfo -j `nproc`

'-j' invalid number '`nproc`' given.

Usage: cmake --build <dir>             [options] [-- [native-options]]
       cmake --build --preset <preset> [options] [-- [native-options]]
Options:
  <dir>          = Project binary directory to be built.
  --preset <preset>, --preset=<preset>
                 = Specify a build preset.
  --list-presets[=<type>]
                 = List available build presets.
  --parallel [<jobs>], -j [<jobs>]
                 = Build in parallel using the given number of jobs. 
                   If <jobs> is omitted the native build tool's 
                   default number is used.
                   The CMAKE_BUILD_PARALLEL_LEVEL environment variable
                   specifies a default parallel level when this option
                   is not given.
  -t <tgt>..., --target <tgt>...
                 = Build <tgt> instead of default targets.
  --config <cfg> = For multi-configuration tools, choose <cfg>.
  --clean-first  = Build target 'clean' first, then b

In [8]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached imageio-2.26.0-py3-none-any.whl (3.4 MB)
     ---------------------------------------- 38.1/38.1 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 222.4/222.4 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 276.2/276.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12076 sha256=9a62b22ea93b8e07dc599ac64e4f6a00297bc36129168c3f99e1cbacfa44e887
  Stored in directory: c:\users\franz\appdata\local\pip\cache\wheels\87\fe\07\21f1a70e1404957105ec892ee37494e251c12441f1d2f6d7cf
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62510 sha256=305e9b5f4dd56edc1d993d808454754ba96d8576c115f7eb797e8a80d1428bc1
  Stored in directory: c:\users\franz\appdata\l

## 6. [LOCAL MACHINE] Run COLMAP on your scene
COLMAP doesn't work on machines without a GUI.

Go to your local machine and follow the [instructions](https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md#preparing-new-nerf-datasets) to run COLMAP from a video or a set of images to generate camera positions from your scene.

After this, you should have an images folder, with the images of your scene, and a `transforms.json` file with the camera information extracted by COLMAP.

## 7. Upload your scene

Mount your google drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

Then upload the `images` folder and the output of COLMAP, `transforms.json`, to your drive. The structure should be similar to the following:
```
/content/drive/MyDrive/nerf_scenes/
└── fox
    ├── images
    │   ├── 00001.jpg
    │   └── 00002.jpg
    └── transforms.json
```



Enter the path to your scene

In [12]:
import os
scene_path = r"data\nerf\fox" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

## 8. Train a model on your scene!

In [13]:
train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.ingp")
!python ./scripts/run.py {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path}

Traceback (most recent call last):
  File "e:\Github\band-gap\instant-ngp\scripts\run.py", line 25, in <module>
    import pyngp as ngp # noqa
ModuleNotFoundError: No module named 'pyngp'


## 9. [LOCAL MACHINE] Generate a camera path

Congrats! You now have a trained nerf checkpoint. Now, in order to generate a video with it, you will need to open it in your local machine with `testbed` and generate a `base_cam.jon` file following these [instructions](https://github.com/NVlabs/instant-ngp#testbed-controls). Remember to launch with the `--no-train` argument so that it doesn't start to train on your PC. Setting up the cameras can make your GUI pretty laggy, you can try to play with the `--height` and `--width` parameters or cropping your scene with the `Crop aabb` options to optimize the performance.

Example command:
```
./build/instant-ngp /data/nerf/fox/2000.ingp
```

After you're done, **upload `base_cam.json` to the root folder of your scene.**

## 10. Render video

Make sure `base_cam.json` exists:

In [ ]:
video_camera_path = os.path.join(scene_path, "base_cam.json")
if not os.path.isfile(video_camera_path):
  raise FileNotFoundError(video_camera_path)

Render the video

In [ ]:
video_n_seconds = 5 #@param {type:"integer"}
video_fps = 25 #@param {type:"integer"}
width = 720 #@param {type:"integer"}
height = 720 #@param {type:"integer"}
output_video_path = os.path.join(scene_path, "output_video.mp4")

!python scripts/run.py {snapshot_path} --video_camera_path {video_camera_path} --video_n_seconds 2 --video_fps 25 --width 720 --height 720 --video_output {output_video_path}
print(f"Generated video saved to:\n{output_video_path}")

22:32:31 INFO     Loading NeRF dataset from
22:32:31 INFO       /content/drive/MyDrive/nerf_scenes/fox/transforms.json
22:32:31 WARNING    /content/drive/MyDrive/nerf_scenes/fox/base_cam.json does not contain any frames. Skipping.
22:32:31 PROGRESS []   0% ( 0/50)  0s/inf22:32:31 PROGRESS []   2% ( 1/50) 0s/1s22:32:31 PROGRESS []   4% ( 2/50) 0s/0s22:32:31 PROGRESS []   6% ( 3/50) 0s/0s22:32:31 PROGRESS []   8% ( 4/50) 0s/0s22:32:31 PROGRESS []  10% ( 5/50) 0s/0s22:32:31 PROGRESS []  12% ( 6/50) 0s/0s22:32:31 PROGRESS []  14% ( 7/50) 0s/0s22:32:31 PROGRESS []  16% ( 8/50) 0s/0s22:32:31 PROGRESS []  18% ( 9/50) 0s/0s22:32:31 PROGRESS []  20% (10/50) 0s/0s22:32:31 PROGRESS []  22% (11/50) 0s/0s22:32:31 PROGRESS []  24% (12/50) 0s/0s22:32:31 PROGRESS []  26% (13/50) 0s/0s22:32:31 PROGRESS []  28% (14/50) 0s/0s22:32:31 PROGRESS []  30% (15/50) 0s/0s22:32:31 PROGRESS []  32% (16/50) 0s/0s22:32:31 PROGRESS []  34% (17/50) 0s/0s22:32:31 PROGRESS []  36% (18/50) 0s/0s22:32:31 PROGRESS []  38% 